# Modelos de clasificacion lineales


En esta notebook veremos como:
1. Levantar un conjunto de datos previamene separado en conjunto de entrenamiento y de test
2. Preprocesar los datos
3. Realizar una seleccion de hiperparametros para cada modelo mediante un Grid Search. En este analisis se consideran los clasificadores: **Regresion Logistica (LogisticRegression), XGBoost, Naive Bayes Multinomial y SVM Lineal**.
4. Seleccionar el mejor clasificador y evaluarlo en el conjunto test.


## 0. Instalacion e importacion de las librerias necesarias

In [ ]:
# Instalamos la biblioteca xgboost
!pip install xgboost

In [ ]:
# Importamos las librerias que necesitamos:
import matplotlib.pyplot as plt
import os
import pandas
import seaborn

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, precision_recall_curve
import matplotlib.pyplot as plt

## 1. Importacion de conjunto de datos etiquetados

In [ ]:
# Creamos variables que indiquen la direccion del conjunto de datos y el nombre de la columna que contiene la etiqueta
direccion_datos = '/users/proyecto_pcd_ssyr/data'
col_etiqueta = 'is_pcd'

In [ ]:
nombre_archivo = os.path.join(direccion_datos, 'annotations/final_pcd_labels.csv')
print('Leyendo datos del archivo', nombre_archivo)
datos = pandas.read_csv(nombre_archivo)

### 1.1 Creacion de los conjunto de entrenamiento y test

In [ ]:
# Levanto el conjunto test creado en la notebook train_test_split
df_test =  pandas.read_csv('test_dataset.csv')[['id_hc', 'edad_consulta', 'sexo', 'especialidad','hccomunidad_original',  'cie10', col_etiqueta]]
# Creo el DataFrame utilizado para el entrenamiento
df_entrenamiento = datos[~datos.id_hc.isin(df_test.id_hc.values)][['id_hc', 'edad_consulta', 'sexo', 'especialidad','hccomunidad_original',  'cie10', col_etiqueta]
]
print('El conjunto cuenta con la siguiente cantidad de instancias totales:', df_entrenamiento.shape[0]+df_test.shape[0])
# Selecciono las columnas que utilizaremos en los modelos
col_atributos = ['id_hc', 'edad_consulta', 'sexo', 'especialidad','hccomunidad_original',  'cie10']
# Creamos los conjuntos de entrenamiento y test:
X_entrenamiento, y_entrenamiento = df_entrenamiento[col_atributos], df_entrenamiento[col_etiqueta]
X_test, y_test = df_test[col_atributos], df_test[col_etiqueta]

## 2. Preprocesamiento de los datos

1. A la columna que contiene el texto libre de la consulta se le aplica la transformacion a matriz *tfidf*
2. La edad de las personas es reescalada entre 0 y 1
3. El sexo registrado, la especialidad y los codigos cie-10 son codificados mediante *OneHotEncoding*.

In [ ]:
# Definimos los pasos de preprocesamiento
preprocessor = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(), 'hccomunidad_original'), 
        ('scaler', MinMaxScaler(), ['edad_consulta']),
        ('onehot_sex', OneHotEncoder(handle_unknown='infrequent_if_exist'), ['sexo']),
        ('onehot_specialty', OneHotEncoder(handle_unknown='infrequent_if_exist'), ['especialidad']),
        ('onehot_cie10', OneHotEncoder(handle_unknown='infrequent_if_exist'), ['cie10'])
    ],
    remainder='drop'
)

# Definimos el flujo de trabajo
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())  # Clasificador de marcador de posición; será reemplazado durante el grid search
])


## 3. Seleccion de hiperparametros y valores de los mismos a testear utilizando *GridSearch* para cada clasificador

In [ ]:

# Preparamos los parametros para el grid search
param_grid = [
    {
        'preprocessor__tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],  # Unigramas, bigramas, y trigramas
        'preprocessor__tfidf__max_features': [30000],  # Limitamos a to 30,000 features
        'classifier': [LogisticRegression()],
        'classifier__C': [0.01, 0.1, 1, 10],  # Regularization strength
        'classifier__max_iter': [100, 200]  # Maximum iterations for convergence
    },
    {
        'preprocessor__tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'preprocessor__tfidf__max_features': [30000],
        'classifier': [XGBClassifier(eval_metric='logloss')],
        'classifier__n_estimators': [50, 100],  # Number of boosting rounds
        'classifier__max_depth': [3, 5],  # Maximum depth of trees
        'classifier__learning_rate': [0.01, 0.1]  # Learning rate
    },
    {
        'preprocessor__tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'preprocessor__tfidf__max_features': [30000],
        'classifier': [MultinomialNB()],
        'classifier__alpha': [0.1, 1.0, 10.0]  # Smoothing parameter
    },
    {
        'preprocessor__tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
        'preprocessor__tfidf__max_features': [30000],
        'classifier': [LinearSVC(dual=True)],
        'classifier__C': [0.01, 0.1, 1, 10],  # Regularization parameter
        'classifier__max_iter': [1000, 2000]  # Maximum iterations
    }
]

# Inicializamos el GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)


grid_search.fit(X_entrenamiento, y_entrenamiento)

# Obtenemos los mejores parámetros y la mejor puntuación
print(f"Mejores hiperparametros: {grid_search.best_params_}")
print(f"Mejor performance de validacion cruzada: {grid_search.best_score_}")

In [ ]:
# Diccionario para almacenar la mejor puntuación para cada clasificador
performance_clasificadores = {}

# Iteramos sobre los resultados
for i in range(len(grid_search.cv_results_['params'])):
    nombre_clasificador = type(grid_search.cv_results_['params'][i]['classifier']).__name__
    performance_media = grid_search.cv_results_['mean_test_score'][i]
    
    # Almacenamos la puntuación más alta para cada clasificador.
    if nombre_clasificador not in performance_clasificadores or performance_media > performance_clasificadores[nombre_clasificador]:
        performance_clasificadores[nombre_clasificador] = performance_media

# Imprimimos el informe
print("Performance máxima alcanzada por cada clasificador:")
for clasificador, performance in performance_clasificadores.items():
    print(f"{clasificador}: {performance:.4f}")

## 4. Seleccion y evaluacion sobre el conjunto test del clasificador que mostro mejor performance en la validacion cruzada

In [ ]:
# Obtenemos el mejor modelo
mejor_modelo = grid_search.best_estimator_

# Realizamos la prediccion sobre el conjunto test
y_pred = mejor_modelo.predict(X_test)

# Print the classification report
print("Reporte de la clasificacion:")
print(classification_report(y_test, y_pred))

In [ ]:
# Ensure that y_test and y_pred are in the appropriate format (binary classification)
# If y_test contains class labels, convert y_pred to probabilities if necessary
if len(set(y_test)) == 2:  # Check if it's a binary classification
    # Get the probability scores for the positive class
    y_pred_proba = mejor_modelo.predict_proba(X_test)[:, 1]

    # Compute ROC AUC
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    print(f"ROC AUC: {roc_auc:.4f}")

    # Compute Precision-Recall AUC
    pr_auc = average_precision_score(y_test, y_pred_proba)
    print(f"Precision-Recall AUC: {pr_auc:.4f}")

    # Compute ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
    plt.figure()
    plt.plot(fpr, tpr, color='blue', label=f'ROC curve (area = {roc_auc:.4f})')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc='lower right')
    plt.show()

    # Compute Precision-Recall curve
    precision, recall, _ = precision_recall_curve(y_test, y_pred_proba)
    plt.figure()
    plt.plot(recall, precision, color='green', label=f'Precision-Recall curve (area = {pr_auc:.4f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.legend(loc='lower left')
    plt.show()
else:
    print("Las métricas ROC y Precision-Recall solo se aplican para la clasificación binaria.")